In [ ]:
import numpy as np
from tqdm import tqdm

# Fraccionamiento
def fracc(n):
    DistF = n / np.sum(n)  # ----> Distribución de probabilidad para escoger el frente
    i = np.random.choice(np.arange(len(n)), p=DistF)   # ----> Escoge un frente basado en la ponderación

    if n[i] > 1:                                       # ----> Solo fraccionar si el frente tiene al menos fuerza 2
        nuevos = np.ones(n[i], dtype=int)              # ----> Crea `n[i]` frentes de valor 1
        n = np.concatenate((n[:i], n[i + 1:], nuevos)) # ----> Concatenar eliminando el frente i

    return n

# Combinación
def comn(n):
    DistF = n / np.sum(n)  # ----> Distribución de probabilidad para escoger los frentes
    i, j = np.random.choice(np.arange(len(n)), size=2, p=DistF)

    # Asegurarse de que i y j sean distintos
    while j == i:
        j = np.random.choice(np.arange(len(n)), p=DistF)

    n[i] += n[j]         # ----> Combina las fuerzas de los frentes
    n = np.delete(n, j)  # ----> Elimina uno de los frentes combinados

    return n

def simulacion(N, m , v, p):
    # N: Fuerza de ataque total
    # m: Número de iteraciones
    # v: Probabilidad de fragmentación
    # p: Cantidad de frames

    ns = np.random.randint(1, N)                # ----> Número aleatorio entre 1 y N de unidades de ataque
    dist = np.random.multinomial(N, [1/ns]*ns)  # ----> Distribución aleatoria inicial
    dist = dist[dist > 0]                       # ----> Filtrar valores mayores que cero

    vecS = [np.unique(dist, return_counts=True)[0]]
    vecF = [np.unique(dist, return_counts=True)[1]]

    # Realizar las m iteraciones considerando probabilidad de fraccionamiento y combinación
    for i in tqdm(range(m), desc="Processing", unit="iteration"):
      if np.random.rand() > v and len(dist) > 1:
        dist = comn(dist)
      else:
        dist = fracc(dist)

      if i % (m // p) == 0:  # Guardar solo cada cierto número de pasos para optimizar
        vecS.append(np.unique(dist, return_counts=True)[0])
        vecF.append(np.unique(dist, return_counts=True)[1])

    return dist, vecS, vecF

N, m, v, p = 1000000, 1000000, 0.01, 10000
dist, vecS, vecF = simulacion(N, m, v, p)

maximo = max(len(S) for S in vecS)
vecS = np.array([np.concatenate((S, np.zeros(maximo - len(S)) ) ) for S in vecS])
vecF = np.array([np.concatenate((F, np.zeros(maximo - len(F)) ) ) for F in vecF])

# Guardar en un archivo de texto
np.savetxt('resultado final.txt', dist)
np.savetxt('Fuerzas.txt', vecS)
np.savetxt('Frecuencias.txt', vecF)

# Imprimir la distribución final y la suma final
print("Distribución final:", dist)
print("Suma final:", np.sum(dist))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from matplotlib.animation import FuncAnimation

# Animación

vecS = np.loadtxt('Fuerzas.txt')
vecF = np.loadtxt('Frecuencias.txt')
vecS = [S[S > 0] for S in vecS]
vecF = [F[F > 0] for F in vecF]
print(len(vecS))
print(len(vecF))

frames = len(vecS)

fig, ax = plt.subplots()

pbar = tqdm(total=frames, desc="Creando animación", unit="frame")

# Función de inicialización
def inic():
    ax.clear()
    ax.set_title("Distribución de la riqueza")
    ax.set_xlabel('Fuerza de ataque')
    ax.set_ylabel('Frecuencia')
    ax.set_xscale('log')
    ax.set_yscale('log')
    return []

def animacion(frame):
    S, fr = vecS[frame], vecF[frame]
    fr = fr/fr[0]

    ax.clear()
    ax.set_title(f"Distribución de la riqueza - Paso {frame * (m // p)}")
    scatter = ax.scatter(S, fr, marker= 'o', color= 'blue')
    ax.set_xlabel('Fuerza de ataque')
    ax.set_ylabel('Frecuencia')
    ax.set_xscale('log')
    ax.set_yscale('log')
    pbar.update(1)
    return scatter,

# Crear la animación
anim = FuncAnimation(
    fig,
    animacion,               # ----> Función de actualización
    frames= frames,          # ----> Número de frames
    init_func=inic,          # ----> Función inicialización
    interval=20,             # ----> Intervalo en milisegundos entre cuadros
    blit=True                # ----> Solo dibujar las partes que han cambiado
)

anim.save("Animacion.mp4", fps=60)